#Set Up

In [25]:
from google.colab import userdata
import os
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

os.environ['GROQ_API_KEY'] = OPENAI_API_KEY

SERPER_API_KEY = userdata.get('SERPER_API_KEY')

os.environ['SERPER_API_KEY'] = SERPER_API_KEY

In [26]:
!pip install crewai openai

In [27]:
from crewai import Agent, Task, Crew, LLM
import warnings
warnings.filterwarnings('ignore')

# Initialize Large Language Model (LLM) of your choice (see all models on our Models page)
llm = LLM(model="openai/gpt-3.5-turbo",api_key=OPENAI_API_KEY)

#crewAI Tools

In [28]:
!pip install crewai-tools

In [29]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

#Creating Agents

In [30]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    llm=llm,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [31]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    llm=llm,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [32]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    llm=llm,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)


In [33]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    llm=llm,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

#Creating Tasks

In [34]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [35]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)

In [36]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)

In [37]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

#Creating the Crew

In [38]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent,
            trading_strategy_agent,
            execution_agent,
            risk_management_agent],

    tasks=[data_analysis_task,
           strategy_development_task,
           execution_planning_task,
           risk_assessment_task],

    manager_llm=llm,
    process=Process.hierarchical,
    verbose=True
)

#Running the Crew

In [39]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [40]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

Output hidden; open in https://colab.research.google.com to view.

In [41]:
print(result)

Here is a comprehensive risk analysis report detailing potential risks and mitigation recommendations for AAPL:

Risk Analysis Report for AAPL Trading Strategies in 2025:

1. Market Risk: Fluctuations in the stock market can significantly impact the value of AAPL stocks. Mitigation Strategy: Diversification of the portfolio to reduce exposure to market volatility.

2. Competition Risk: Intense competition in the tech industry can affect AAPL's market position and profitability. Mitigation Strategy: Continuous innovation and product differentiation to stay ahead of competitors.

3. Regulatory Risk: Changes in regulations, trade policies, or legal issues could impact AAPL's operations. Mitigation Strategy: Regular monitoring of regulatory changes and proactive compliance measures.

4. Technology Risk: Rapid technological advancements may render AAPL's products obsolete or less competitive. Mitigation Strategy: Investment in Research and Development to stay at the forefront of technologic

In [42]:
from IPython.display import Markdown
Markdown(result.raw)

Here is a comprehensive risk analysis report detailing potential risks and mitigation recommendations for AAPL:

Risk Analysis Report for AAPL Trading Strategies in 2025:

1. Market Risk: Fluctuations in the stock market can significantly impact the value of AAPL stocks. Mitigation Strategy: Diversification of the portfolio to reduce exposure to market volatility.

2. Competition Risk: Intense competition in the tech industry can affect AAPL's market position and profitability. Mitigation Strategy: Continuous innovation and product differentiation to stay ahead of competitors.

3. Regulatory Risk: Changes in regulations, trade policies, or legal issues could impact AAPL's operations. Mitigation Strategy: Regular monitoring of regulatory changes and proactive compliance measures.

4. Technology Risk: Rapid technological advancements may render AAPL's products obsolete or less competitive. Mitigation Strategy: Investment in Research and Development to stay at the forefront of technological innovation.

5. Supply Chain Risk: Disruptions in the supply chain can affect production and distribution of AAPL products. Mitigation Strategy: Establishing alternative sourcing options and maintaining strong supplier relationships.

6. Financial Risk: Economic downturns, currency fluctuations, or interest rate changes can impact AAPL's financial performance. Mitigation Strategy: Robust financial planning and hedging strategies to manage financial risks.

7. Reputation Risk: Negative publicity, product recalls, or scandals can harm AAPL's brand reputation. Mitigation Strategy: Proactive communication, transparency, and strong corporate governance.

8. Cybersecurity Risk: Data breaches or cyber attacks could compromise sensitive information and damage customer trust. Mitigation Strategy: Investment in cybersecurity measures, regular security audits, and employee training.

9. ESG Risk: Environmental, Social, and Governance factors can influence investor perception and impact AAPL's stock value. Mitigation Strategy: Sustainability initiatives, transparent ESG reporting, and adherence to ethical business practices.

10. Geopolitical Risk: Political instability, trade disputes, or global events can create uncertainties for AAPL's business operations. Mitigation Strategy: Diversification of global markets and close monitoring of geopolitical developments.

This risk analysis report provides a comprehensive overview of potential risks and suggests mitigation strategies to protect AAPL's interests and enhance long-term sustainability in the ever-changing market environment.